# MongoDB Childcare Database
This databse is a collection of data about children, staff and their events in a daycare.

You will notice the first way we have it setup is very relational. 

We have a table for `children`, `staff` and `events`. We have a relationship between children, staff and events via the `childId` and the `staffId` on the events table linking back to the `children` and `staff` tables. 

```mermaid
erDiagram
    children ||--o{ dailyEvents : has
    staff ||--o{ dailyEvents : manages

    children {
        ObjectId _id
        string firstName
        string lastName
        date dateOfBirth
        datetime enrollmentDate
        array allergies
        object emergencyContact
    }

    staff {
        ObjectId _id
        string firstName
        string lastName
        date hireDate
        string role
        array credentials
    }

    dailyEvents {
        ObjectId _id
        ObjectId childId
        ObjectId staffId
        datetime timestamp
        string eventType
        string details
        string notes
    }
```


In [47]:
from datetime import datetime, timedelta
import time
from pprint import pprint
import os
from pymongo import MongoClient
from pymongo.synchronous.command_cursor import CommandCursor
from typing import List
from dotenv import load_dotenv

# check to see if .env file exists and load it
if os.path.exists('../.env'):
    load_dotenv()
else:
    raise FileNotFoundError(".env file not found. Did you copy the .env.example file and rename it to .env?")

DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')


client = MongoClient(f'mongodb://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}')

db = client[DB_NAME]

def get_time_for_filters(days_to_go_back: int = 3) -> datetime:

    result = db['dailyEvents'].find(
        filter={},
        sort={'timestamp': -1},
        limit=1
    )   

    ts = result[0]['timestamp']
    # get events from 14 days ago(figure 2 weeks worth of events)
    ts = result[0]['timestamp'] - timedelta(days=days_to_go_back)
    # Go to the start of the day
    return ts.replace(hour=0, minute=0, second=0, microsecond=0)


events_since_time = get_time_for_filters(days_to_go_back=14)

# leave this manual setting here just in case we need to go back to a specific date
# events_since_time = datetime.strptime("12-01-2024 00:00:00", "%m-%d-%Y %H:%M:%S")



In [74]:
def get_events_and_children_since_time() -> CommandCursor:
    """
    Retrieves daily events and associated child information since a specified time.

    This function performs an aggregation pipeline on the 'dailyEvents' collection:
    1. Matches events with timestamps greater than or equal to 'events_since_time'.
    2. Looks up corresponding child information from the 'children' collection.
    3. Projects specific fields from both events and child information.

    The function also measures and prints the execution time and number of documents returned.

    Returns:
        CommandCursor: A cursor to iterate over the matching events with child information.
    """

    result = db['dailyEvents'].aggregate([
        {
            '$match': {
                'timestamp': {
                    '$gte': events_since_time
                }
            }
        }, {
            '$lookup': {
                'from': 'children', 
                'localField': 'childId', 
                'foreignField': '_id', 
                'as': 'childInfo'
            }
        }, {
            '$project': {
                'notes': 1, 
                'details': 1, 
                'eventType': 1, 
                'childId': 1, 
                'staffId': 1, 
                'timestamp': 1, 
                'childInfo.firstName': 1, 
                'childInfo.lastName': 1
            }
        }
    ])
    
    # result is just a cursor and doesn't return any data till you iterate over it
    print_num_events(result)


    return result


def print_num_events(result: CommandCursor) -> List:
    """
    Prints the execution time and number of documents returned from a CommandCursor.

    This function executes the query represented by the CommandCursor, measures the
    execution time, and prints both the time taken and the number of documents returned.

    Args:
        result (CommandCursor): The CommandCursor object representing the query to be executed.

    Returns:
        List: Returns the data returned by the query.
    """

    # time before we run the query
    start_time = time.time()

    data = list(result) # this actually runs the query and returns the data
    
    # time taken to run the query
    end_time = time.time()
    
    execution_time = end_time - start_time
    print(f"Execution time: {execution_time} seconds")
    num_of_docs = len(data)
    print(f"Number of documents: {num_of_docs}")

    return data


def run_explain_on_pipeline(pipeline, collection_name: str) -> dict:
    """
    Runs an explain command on the given pipeline for the specified collection.

    This function executes an explain command to analyze the performance of an aggregation
    pipeline on a specific collection. It provides detailed execution statistics for the
    pipeline, which can be useful for query optimization.

    Args:
        pipeline (list): The aggregation pipeline to explain.
        collection_name (str): The name of the collection to run the explain on.

    Returns:
       Dict: The result of the explain command.
    """
    
    explain_command = {
    "explain": {
        "aggregate": collection_name,
        "pipeline": pipeline,
        "cursor": {}
    },
    "verbosity": "allPlansExecution"
}
    return db.command(explain_command)
    

def get_events_since_time(explain: bool = False) -> CommandCursor:
    """
    Retrieves daily events from the database since a specified time.

    This function queries the 'dailyEvents' collection for events with timestamps
    greater than or equal to the 'events_since_time'. It measures and prints the
    execution time and the number of documents returned.

    Returns:
        CommandCursor: A cursor to iterate over the matching events.
    """

    pipeline = [
        {
            '$match': {
                'timestamp': {
                    '$gte': events_since_time
                }
            }
        }
    ]
    
    if explain:
        results = run_explain_on_pipeline(pipeline, "dailyEvents")
        used_collscan = False
        used_index = False
        index_name = ""
        winning_plan = results["queryPlanner"]["winningPlan"]
        if winning_plan["stage"] == "COLLSCAN":
            used_collscan = True
        if 'inputStage' in winning_plan: # this might not be enough we shall see
            if winning_plan['inputStage']['stage'] == "IXSCAN":
                used_index = True
                index_name = winning_plan['inputStage']['indexName']

        
        parsed_results = {
            "execution_millis": results["executionStats"]["executionTimeMillis"],
            "total_docs_examined": results["executionStats"]["totalDocsExamined"],
            "used_collscan": used_collscan,
            "used_index": used_index,
            "index_name": index_name,
            "total_keys_examined": results["executionStats"]["totalKeysExamined"],
            "total_docs_examined": results["executionStats"]["totalDocsExamined"]
        }
        
        return parsed_results, results
    
    result = db['dailyEvents'].aggregate(pipeline)

    
    # result is just a cursor and doesn't return any data till you iterate over it
    print_num_events(result)

    return result

# Limit the number of documents returned

Limit the events by a certain time range, example last 7 days.

## What problems could we have here?

We are not using any index here. If we were to add an index I bet this would be even faster.

## Examining the results

Running the cell below we will run a query to get all of the events since 7 days ago and get the explain plan for the query.

You should get results similar to the following. Notice that we are using a collscan and it is taking an estimated 459ms to run the query. Keep in mind that this is just an estimate and your results might be different.

```json
{
    'execution_millis': 459,
    'index_name': '',
    'total_docs_examined': 2000000,
    'used_collscan': True,
    'used_index': False
 }
```

In [39]:
# By setting explain=True we can get the explain plan for the query. 
parsed_results, results = get_events_since_time(explain=True)
pprint(parsed_results)


{'execution_millis': 459,
 'index_name': '',
 'total_docs_examined': 2000000,
 'used_collscan': True,
 'used_index': False}


In [ ]:
# Feel free to run the query to see the results.
get_events_since_time()

# Adding an index

Anyone who knows SQL knows that creating an index can help speed up queries. 

What do we add a index to in the above query?
When adding an index we should add an index to fields that we are using in a `where clause` or in MongoDB's aggregation pipeline the `$match` or in a `find({'field': 'value'})` on the field. Also we should look at indexes on fields that are used in a `sort`.

In the query above we are matching on the timestamp field, let's add an index to that. 

In MongoDB we can add an index to a field by using the `create_index` method. You also provide a parameter for accending or descending sort order but which do we use? 

My gut would say we care about the most recent events but there is this note in the docs that stats indexes using descending order can cause performance issues and only use ascending order for indexes. https://www.mongodb.com/docs/manual/core/indexes/create-index/#example so We will start with that, test it, remove it and check descending order and see if there is a difference.



In [40]:
# Add index to the timestamp field in ascending order
db['dailyEvents'].create_index([('timestamp', 1)])

'timestamp_1'

# Running the query with the index ascending

Run the query in a seperate cell to not have the index creation influence the run time

## Getting the explain plan with explain=True
First lets run with an explain to make sure we are using the index

You should get results similar to the following. Notice that we are using a index and it is taking an estimated 1407 to run the query. Continue on to see why this is.


```json
{
    'execution_millis': 1407,
    'index_name': 'timestamp_1',
    'total_docs_examined': 990580,
    'used_collscan': False,
    'used_index': True
 }
```



In [42]:
# First lets clear the query plan cache
db.command({"planCacheClear": "dailyEvents"})
# By setting explain=True we can get the explain plan for the query. 
parsed_results, results = get_events_since_time(explain=True)
pprint(parsed_results)

{'execution_millis': 1331,
 'index_name': 'timestamp_1',
 'total_docs_examined': 990580,
 'used_collscan': False,
 'used_index': True}


## Why are the executions times more?

We are using an index, why are the execution times more?!?! Notice we are returning 990580 in my example, yours maybe different depending on your dataset. As long as you are returning more then 600_000 or 30% of 2_000_000 documents then you should see the same issue.

Why would this happen?

When the query would return a large portion of the collection (typically >30% of documents):
- The index scan plus document lookup becomes more expensive than a simple collection scan
- MongoDB has to look up each document in the index and then fetch the actual document
- If you look at the full query plan you will see multiple stages for the query.

<
## So what does this mean

This means that this is why we should always test our queries after adding an index. Just because we added an index doesn't mean that we are actually improving anything.

Always follow these steps:
- Run the query with an explain. Document how long the query plan stats it will run via the executionTimeMillisEstimate
- Calculate if the number of rows that is going to be returned is greater than 30%. If so maybe an index doesn't make sense. 
- Add the index you plan to use
- Run the query explain again. Make sure you document the total executionTimeMillisEstimate not just the executionTimeMillisEstimate for each stage.
- If the index makes it worse remove the index. Feel free to try another index just as we are going to down below for descending order just to make sure.
- If you have trouble reading the explain plan recreate the query in MongoDB Compass and run the explain there. The visual is much more readable.

In [ ]:
get_events_since_time()

# Adding a descending index

Just for fun, lets add a descending index on the timestamp field to see what happens. 



In [43]:
# Remove the index on the timestamp field
try:
    db['dailyEvents'].drop_index('timestamp_1')
except Exception:
    print("Index does not exist")
# For fun maybe we clear the query plan cache
db.command({"planCacheClear": "dailyEvents"})

# Recreate the index on the timestamp field in descending order
db['dailyEvents'].create_index([('timestamp', -1)])


'timestamp_-1'

In [ ]:
# First lets clear the query plan cache
db.command({"planCacheClear": "dailyEvents"})
# By setting explain=True we can get the explain plan for the query. 
parsed_results, results = get_events_since_time(explain=True)
pprint(parsed_results)

# Removing the descending index
You shouldn't see any difference here, maybe a couple of milliseconds faster but still way slower than not using an index so lets remove it and carry on.

In [46]:
# Drop the descending order index
try:
    db['dailyEvents'].drop_index('timestamp_-1')
except Exception:
    print("Index does not exist")

# What bout on a smaller window of time

Let's see if the index actually makes a difference on this dataset. We will do this by running the query with a $gte with looking for 3 days of data. 

First lets create the ascending index, then run the query and explain for 3 days of data.

I am getting results back similar to the following:

```json
{
    'execution_millis': 388,
    'index_name': 'timestamp_1',
    'total_docs_examined': 256847,
    'used_collscan': False,
    'used_index': True
 }
```

Without and index we are using a collscan and it is taking an estimated 459ms to run the query. This is 388 to 400 with the index. In my opnion it may not be worth it with this level of data.

In [63]:
# create the index on the timestamp field in ascending order
db['dailyEvents'].create_index([('timestamp', 1)])

'timestamp_1'

In [ ]:
# reset the time we are querying from
events_since_time = get_time_for_filters(days_to_go_back=3)

# By setting explain=True we can get the explain plan for the query. 
parsed_results, results = get_events_since_time(explain=True)
pprint(parsed_results)

In [61]:
# drop the index on the timestamp field
try:
    db['dailyEvents'].drop_index('timestamp_1')
except Exception:
    print("Index does not exist")

# Let's explode our dataset

This is a small dataset so lets see if we can get a feel for how MongoDB works. We will create a new dataset with the same amount of children and staff members but greatly increase the number of events. We will keep the same number of events within the last 30 days but increase the number of older events overall.

First we need to drop the collections

In [62]:
# Drop the children, staff, and dailyEvents collections
db['children'].drop()
db['staff'].drop()
db['dailyEvents'].drop()


Next we need to generate the new dataset. Open your .env file and set GENERATE_OLD_EVENTS=true

Then open your terminal to the root of this repo and run the following command: `uv run generate_data.py`

You should see an output similar to the following:
```bash
Inserting Children Data
Inserting Staff Data
Inserting Events Data
Generating old events
```

Then after a while you will see it start inserting the data. This could take a while as it is a lot of data it is generating.

Once this is done you should have the same database setup you had before but with

- 10k children
- 100 staff members
- 4,000,000 events. 2 million of which are in the last 30 days and 2 million of which are older. Lets see how these queries perform with this new dataset.

First lets make sure we have a clean slate of indexes

Then lets query for 14 days worth of data without an index

In [64]:
# List all indexes in the dailyEvents collection except for _id_
indexes = db['dailyEvents'].list_indexes()
non_id_indexes = [index for index in indexes if index['name'] != '_id_']
print("Existing indexes (excluding _id_):")
for index in non_id_indexes:
    print(f"Name: {index['name']}")
    db['dailyEvents'].drop_index(index['name'])


Existing indexes (excluding _id_):
Name: timestamp_1


## Running the query without an index

The results are a little more interesting this time but well within what we would expect. We are using a collscan and it is taking an estimated 897ms to run the query over 4_000_000 documents.

```json
{
    'execution_millis': 897,
    'index_name': '',
    'total_docs_examined': 4000000,
    'used_collscan': True,
    'used_index': False
 }
```

In [ ]:
# Query for 14 days with of data

# reset the time we are querying from
events_since_time = get_time_for_filters(days_to_go_back=14)

# By setting explain=True we can get the explain plan for the query. 
parsed_results, results = get_events_since_time(explain=True)
pprint(parsed_results)

# Readd and query with an index

Lets add the index on the timestamp field in ascending order then requery for 14 days worth of data and see if we see any difference.

In [66]:
# Recreate the index on the timestamp field in ascending order
db['dailyEvents'].create_index([('timestamp', 1)])

'timestamp_1'

You should see results similar to the following. Notice that we are using a index and it is taking an estimated 1469ms to run the query. You can also see the total number of documents that were scanned and the total number of keys that were scanned are the same. This means that we are getting every document from the index.

```json
{
    'execution_millis': 1469,
    'index_name': 'timestamp_1',
    'total_docs_examined': 994169,
    'total_keys_examined': 994169,
    'used_collscan': False,
    'used_index': True
 }

In [ ]:
# Requery for 14 days with of data

# reset the time we are querying from
events_since_time = get_time_for_filters(days_to_go_back=14)

# By setting explain=True we can get the explain plan for the query. 
parsed_results, results = get_events_since_time(explain=True)
pprint(parsed_results)

# So this is neat and all but....

There has to be a better way to do this. Odd's are you want just the events for a specific child or staff memeber so you could create compound indexes on the date and the id's. However, let's look at a few more options.

## Where did we go wrong?

Odds are we jumped into designing the database without thinking about how we would query for data. Classic rookie mistake with a document database vs just doing what we do with a relational database. When we design for a document database we should design for how we plan to use the data.

### Embeded Data

Let's think about how our users will be accessing the data. Our customers are daycare centers but their customers are parents of children. They want to be able to see what their children have done in the daycare that day. Lets ask ourselves a series of questions:
- Do we always need this data when we query for a child?
- What's the date range we need to query for?
- Is the data unbounded?
- Can we store some embeded but also have it stored in a separate collection for other access?


For this example let's say parent's want to be able to open the app and see everything their child has done today and they want it to be as fast as possible, however, if they want to run reports and see data for other days they are ok with a small load time. So lets keep our daily events collection as simple as possible. This is known as the subset pattern.

Let's also update our children collection to include a field for the clients location they are at. This is an array of locations that has a start_date and and end_date.

This will allow us to easily query for their current location and previous locations of when they were there.

So let's redesign our children collection:

```
{
    ObjectId _id
    string firstName
    string lastName
    date dateOfBirth
    datetime enrollmentDate
    array customerLocation [{
        ObjectId _id
        string name
    }]
    array allergies ['nuts', 'dairy', 'gluten', 'soy', 'eggs', 'fish']
    object emergencyContact {
        string name
        string phone
        string relationship
    }
    array dailyEvents [{
        datetime timestamp
        string eventType
        string details
        string notes
    }]
}
```

Adding a customer location field also means we probably need to have a location collection to store all of the locations and their information.

```
locations{
    ObjectId _id
    string name
    string address
    string city
    string state
    string zip
    string country
    contactInfo {
        string phone
        string email
    }
    string website
}
```

#### Handling old events

So what do we do with old events. Do we need to keep them? Yes we do as parents want to be able to go back and see what their child did in the daycare on previous days.

To handle this we will keep our dailyEvents collection. Every day we will run a cron to move events for each child and insert them into the dailyEvents collection. This way we can keep track of all of the events for each child.

TBD: Update the dailyEvents collection. We should follow the bucket pattern for dailyEvents per location. Inside each location we should have a bucket for each day. This way we can keep track of all of the events for each child. This will also allow us to index the data easier.
